In [103]:
import torch
from torch import nn
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import timeit
import scipy.stats
import numpy as np
from scipy.stats import norm



In [104]:
class MyLogNormalSum(nn.Module):
    def __init__(self):
        super().__init__()
        self.mu = torch.nn.Parameter(torch.zeros(1,dtype=torch.float)) # mean
        self.Sigma = torch.nn.Parameter(torch.ones(1,dtype=torch.float)) # variance

    def forward(self, x):
        y_pred = 0
        for u in x:
            y_pred += (-torch.log(2*np.pi*self.Sigma)/2 -(u-self.mu)**2/(2*self.Sigma))
        return y_pred


In [132]:
mydevice = torch.device("cpu") # "cpu" or "mps"

# model to be trained
model = MyLogNormalSum().to(mydevice)

# number of batches
nbatch = 1000

# learning rate
learn = 1e-3

# batch size
b_size = 300

# loss function
# loss_fn = nn.MSELoss()

# parameters used to generate training data
goalmu = .314
goalSigma = 1.5 # variance, not std dev

# training data features
# x = torch.normal(goalmu, np.sqrt(goalSigma), size=(nbatch,b_size))
x = goalmu + np.sqrt(goalSigma)*torch.randn((nbatch,b_size))

# generate labels
# def Y(x):
#     return(torch.transpose(torch.matmul(goaltheta,torch.transpose(x,0,1)),0,1))

optimizer = torch.optim.Adam(model.parameters(), lr=learn, maximize=True)


def runloop():
    for i in range(nbatch):
        optimizer.zero_grad()
        llhood = model(x[i])
        llhood.backward()
        optimizer.step()
        if i%(nbatch/10)==0:
            print("-" * 10)
            print(i,llhood)
res = timeit.timeit(runloop, number=1)
print('time = ',res)                        
print('trained = ',model._parameters)

# print('target = ',goaltheta)
# print('mismatch = ', nn.functional.mse_loss(list(model.parameters())[0].data, goaltheta))



----------
0 tensor([-477.5761], grad_fn=<AddBackward0>)
----------
100 tensor([-477.8777], grad_fn=<AddBackward0>)
----------
200 tensor([-486.7375], grad_fn=<AddBackward0>)
----------
300 tensor([-460.2825], grad_fn=<AddBackward0>)
----------
400 tensor([-479.4442], grad_fn=<AddBackward0>)
----------
500 tensor([-503.1478], grad_fn=<AddBackward0>)
----------
600 tensor([-476.3310], grad_fn=<AddBackward0>)
----------
700 tensor([-470.0957], grad_fn=<AddBackward0>)
----------
800 tensor([-470.8931], grad_fn=<AddBackward0>)
----------
900 tensor([-470.9055], grad_fn=<AddBackward0>)
time =  25.567295042001206
trained =  OrderedDict([('mu', Parameter containing:
tensor([0.3147], requires_grad=True)), ('Sigma', Parameter containing:
tensor([1.4457], requires_grad=True))])


In [67]:
a=model.parameters()
a.__next__()

Parameter containing:
tensor(1.9000, requires_grad=True)

In [27]:
a.__next__()

Parameter containing:
tensor(0.0400, requires_grad=True)

In [63]:
x = torch.tensor([1.,2.,3.],requires_grad=True)
y = torch.tensor([.5,.3,.5],requires_grad=False)
t = torch.tensor([0,1,2],requires_grad=False)

z = x*y*(2+t)
loss = torch.sum(z)
loss.backward()

In [66]:
print(x.grad)

tensor([1.0000, 0.9000, 2.0000])


In [60]:
y = torch.tensor([2.,2.,2.],requires_grad=False)
z = x*y*3
loss = torch.sum(z)

In [72]:
model._parameters

OrderedDict([('mu',
              Parameter containing:
              tensor(1.9000, requires_grad=True)),
             ('sigma',
              Parameter containing:
              tensor(0.0400, requires_grad=True))])